### Multiple Stock Prediction

### Summary:

The three stocks chosen for predicting daily % returns on open price are Apple, IBM and Microsoft. 

This file includes:

1. Importing the datasets downloaded from WRDS and other resources (source dataset).

2. Performing feature engineering to make custom indicators.


In [ ]:
'''!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz #this code was used to install talib on colab
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib'''

In [ ]:
# Importing all the relevant libraries
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os
import seaborn as sns
from datetime import datetime
import talib
from talib import RSI, BBANDS, MACD, ATR

import pywt
from scipy.special import ndtr
import tqdm
from functools import partial
from scipy.stats import norm
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from pandas.plotting import scatter_matrix
import seaborn as sns
from matplotlib import pyplot

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.compose import ColumnTransformer
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

### 1. Importing all the relevant datasets:

In [ ]:
#Daily stock data
df_prices = pd.read_csv("Daily_stock.csv")   
df_prices.date = pd.to_datetime(df_prices.date)

df_sectors = pd.read_csv("GIC_Sectors.csv")
df_sectors.datadate = pd.to_datetime(df_sectors.datadate)

#DIX index data
df_DIX=pd.read_csv("DIX_GEX_SP500.csv")
df_DIX.date = pd.to_datetime(df_DIX.date)

#VIX index data
df_vix = pd.read_csv("VIX.csv")  
df_vix.Date=pd.to_datetime(df_vix.Date)

df_sectors.rename(columns={"gsector":"GSECTOR","LPERMNO":"PERMNO",'datadate':'date'}, inplace=True)
df_vix.rename(columns={'Date':'date'}, inplace=True)

df_sectors.drop(columns=["iid","tic"], axis=1, inplace=True)
df_sectors.drop_duplicates(["PERMNO"], inplace=True)
df2 = pd.merge(df_prices,df_sectors,on=['PERMNO','date'],how='left') #Add GICSECTOR to price data
df3=pd.merge(df2,df_DIX,on=['date'],how='left')
df4=pd.merge(df3,df_vix,on=['date'],how='left')

df4.drop(['GVKEY','GSECTOR','HSICMG'],axis=1,inplace=True)
df4["PRC"] = df4.PRC.abs() #a negative sign is sometimes used as a marker, filter it out

df4.set_index(["TICKER","date"], inplace=True)
df4.sort_index(level=['TICKER','date'], ascending=True, inplace=True)
df4.drop(['PERMNO'],axis=1,inplace=True)
df4.head(5)

PRC       VOL        BID        ASK    OPENPRC  \
TICKER date                                                               
AAPL   2011-01-03  329.57001  16438280  329.70999  329.72000  325.64001   
       2011-01-04  331.29001  11444072  331.29001  331.29999  332.44000   
       2011-01-05  334.00000   9486655  333.97000  334.00000  329.54999   
       2011-01-06  333.73001  11138250  333.64999  333.79001  334.71899   
       2011-01-07  336.12000  11645048  336.12000  336.26001  333.98999   

                     sprtrn  price  dix  gex   vixo   vixh   vixl    vix  
TICKER date                                                               
AAPL   2011-01-03  0.011315    NaN  NaN  NaN  17.94  17.95  16.91  17.61  
       2011-01-04 -0.001313    NaN  NaN  NaN  17.34  18.24  17.33  17.38  
       2011-01-05  0.005007    NaN  NaN  NaN  17.81  17.95  16.86  17.02  
       2011-01-06 -0.002123    NaN  NaN  NaN  16.80  17.56  16.79  17.40  
       2011-01-07 -0.001845    NaN  NaN  NaN  17.31  18.07  16.57  17.14

In [ ]:
df4.reset_index(inplace=True)

### 2. Importing all the relevant datasets for making put-call and put-parity indicators

In [ ]:
dailystock= pd.read_csv("Daily_stock.csv", index_col=['date'], parse_dates=True)
dailystock.index.name="date"

strikeprice = pd.read_csv("strike.csv", index_col=['date'], parse_dates=True)
strikeprice.index.name="date"

yield_curve= pd.read_csv("yield_curve.csv", index_col=['date'], parse_dates=True)
yield_curve.index.name="date"

volume = pd.read_csv("option_volume.csv", index_col=['date'], parse_dates=True)
volume.index.name="date"

### 3. Designing custom indicators

### The following custom indicators have been developed:

#### 1. `Bollinger bands`

- They are used to determine the fluctuations in the volatility of stock price by calculating the standard deviation below and above the simple moving average price. The upper and lower bands are calculated and used together to determine the price highs and lows. It can detect any irregularities in the trading and is reliable for long term prediction.   ​

- We make a function that gives us a dataframe with higher and lower bands using groupby() and apply() functions. The TA-lib BBANDS gives three values that will be used in the function. In order to gain insight on the current value as per the latest trends in volatility, a log of % difference between upper/lower bands and stock price is taken .

#### 2. `Moving Average Convergence Divergence(MACD)`

- It is a momentum indicator reflecting the relationship between two moving averages of a price. It can be used to identify any changes in the trend curve thereby giving an idea of buy/sell. We design it in a way that it signifies the difference between short and long term EMA.

#### 3. `Put-Call Ratio`

- Helps identify the ongoing mood of the market. 'Put' signifies the right to sell a stock at prespecified price while 'Call' signifies the right to buy for the stock. Extreme values of the ratio can indicate a highly bearish or bullish sentiment of the market.

- In our case, we import the volume data from WRDS where the cp_flag is used to calculate the put call ratio.

#### 4. `Put-Call Parity`

- Represents the relationship between put and call options with same asset/strike prices that are European. Their divergence helps derive insight about the future profits from stock.

- The put-call parity is programmed as follows:

- The strike price for 10 days along with the put-call flag are joined with the daily stock data. The resulting df is then joined with yield curve data for 7 days. The present strike price is calculated as: No.of call/np.exp(Rate per day*No.of days)

- The put call parity is calculated as the difference between PRC and present strike price. The null values are imputed with forward fill.

#### 5. `Shifted lagged returns`

- This indicator was used to verify stock trends and trend changes. The returns are shifted by 1,2,3..10 days.

#### 6. `RSI`

- A very good technical indicator widely used to determine the deviations in price. It indicates signals regarding bullish/bearish momentum of a security’s price. ​The time frame used in our case is 14 days. We use talib library for this indicator.

#### 7. `ADX`
- Indicates the trend’s strength which the current security is encompassing. ​ We use talib library for this indicator.

#### 8. `Parabolic SAR`

- Indicates the security's trend direction and potential price turning point. ​We use talib library for this indicator.

#### 9. `Exponential Moving Average(EMA)`
-  The following code calculates the Exponential Moving Average (EMA) for a given stock and time interval. EMA places more emphasis on the latest stock information (the latest information has the biggest impact on the trend) in contrast too a simple moving average,which distributes the same wight to all the instances in the time interval. Since we are predicting % Daily returns on stock prices, the ideal dates for EMA would be 10, 21, and 50 days.
References: https://tradeciety.com/how-to-use-moving-averages/



In [ ]:
#bollinger bands
def compute_bb(close):
  high, mid, low = BBANDS(close, timeperiod=20)
  return pd.DataFrame({'bb_high': high, 'bb_low': low}, index=close.index)
  
#MACD
def compute_macd(close):
  macd = MACD(close)[0]
  return (macd - np.mean(macd))/np.std(macd)

#PUT-CALL PARITY
def put_call_parity(yield_curve):
  df_temp =  pd.pivot_table(strikeprice,values='strike_price',index='date',columns=['cp_flag'])
  df_temp['days']=10
  df_temp = dailystock.join(df_temp)
  yield_curve=yield_curve[yield_curve["days"]==7]
  yield_curve.drop(["days"], axis=1, inplace=True)
  df_temp = df_temp.join(yield_curve)
  df_temp["rate_per_day"]=df_temp.rate/365
  df_temp["Present_strike_price"] = df_temp.C/np.exp(df_temp.rate_per_day*df_temp.days)
  df_temp["PutCallParity"] = df_temp.PRC - df_temp.Present_strike_price
  df_temp.fillna(method = "ffill", inplace=True)
  return df_temp

#PUT-CALL RATIO
def put_call_ratio(df_temp):
  df_vol =  pd.pivot_table(volume,values='volume',index='date',columns=['cp_flag'])
  df_vol["PutCallRatio"] = df_vol.P/df_vol.C
  df_vol.drop(["C","P"], axis=1, inplace=True)
  df_temp=df_temp.join(df_vol)
  df_temp.fillna(method = "ffill", inplace=True)
  return df_temp

In [ ]:
class FeatureEngineering():
  def __init__(self, 
               add_rsi= True,
               add_bb = True,
               add_macd = True,
               add_ema = True,
               ema_dates = [10, 21, 50],
               add_adx = True,
               adx_timeperiod=10,
               add_sar = True,
               sar_acceleration=0.2,
               sar_maximum=0.2,
               add_lagreturns=True
               ):
    
    self.add_rsi = add_rsi
    self.add_bb = add_bb
    self.add_macd = add_macd
    self.add_ema = add_ema
    self.ema_dates = ema_dates
    self.add_adx = add_adx
    self.adx_timeperiod = adx_timeperiod
    self.add_sar = add_sar
    self.sar_acceleration = sar_acceleration
    self.sar_maximum = sar_maximum


  def fit(self, df):
    return self
 
  #Need to pass the df excluding the target  
  def transform(self, df):
    df['close']   = df.groupby('TICKER')['PRC'].shift(1)
    df['vol']     = df.groupby('TICKER')['VOL'].shift(1)
    df['high']    = df.groupby('TICKER')['ASK'].shift(1)
    df['low']     = df.groupby('TICKER')['BID'].shift(1)
    
    #Computing Lagged returns
    for i in df.TICKER.unique().tolist():
        idx = df[df['TICKER'] == i].index.tolist() 
        for n in list(range(0,10)):
          name = 'lag_ret' + str(n)
          df.loc[idx,name] =  df['OPENPRC'].pct_change(1).shift(n).fillna(0)
    
    #Computing RSI
    if self.add_rsi:
      df['rsi'] = df.groupby('TICKER').close.apply(RSI)
    
    #Computing bollinger bands
    if self.add_bb:
      df = df.join(df.groupby('TICKER').close.apply(compute_bb))
      
      df['bb_high'] = df.bb_high.sub(df.close).div(df.bb_high).apply(np.log1p)
      df['bb_low']  = df.close.sub(df.bb_low).div(df.close).apply(np.log1p)

    #Computing MACD
    if self.add_macd:
      df['macd'] = (df.groupby('TICKER', group_keys=False).close.apply(compute_macd))
    
    # Computing EMA
    if self.add_ema:
      # Looping through all the different combinations of stock and time intervals
      for i in df.TICKER.unique().tolist():
        # Time Interval which we will calculate EMA for
        for j in self.ema_dates:          
          idx = df[df['TICKER'] == i].index.tolist() 
          # Using Talib library to calculate the EMA 
          df.loc[idx,"EMA For " + str(j) + " Days"] = talib.EMA(df[df["TICKER"]== i].PRC, timeperiod = j)

      df[["EMA For 10 Days","EMA For 21 Days", "EMA For 50 Days"]].fillna(0)
    
    #Computing ADX
    if self.add_adx: 
      for i in df.TICKER.unique().tolist():
        idx = df[df['TICKER'] == i].index.tolist() 
        df.loc[idx,'ADX'] = talib.ADX(df[df["TICKER"]== i].high, df[df["TICKER"]== i].low, df[df["TICKER"]== i].close, timeperiod = self.adx_timeperiod)
    
    #Computing SAR
    if self.add_sar: 
      for i in df.TICKER.unique().tolist():
        idx = df[df['TICKER'] == i].index.tolist() 
        df.loc[idx,'SAR'] = talib.SAR(df[df["TICKER"]== i].high, df[df["TICKER"]== i].low, acceleration=self.sar_acceleration, maximum=self.sar_maximum)

    df_temp=put_call_parity(yield_curve)
    df_temp=put_call_ratio(df_temp)
    df_temp.reset_index(inplace=True)

    df2=pd.merge(df,df_temp[['TICKER','date','PutCallParity','PutCallRatio']],on=['date','TICKER'],how='left')

    return df2

In [ ]:
#use the class from above to transform the df
df4 = FeatureEngineering().transform(df4)
df4

,TICKER,date,PRC,VOL,BID,ASK,OPENPRC,sprtrn,price,dix,...,bb_high,bb_low,macd,EMA For 10 Days,EMA For 21 Days,EMA For 50 Days,ADX,SAR,PutCallParity,PutCallRatio
0,AAPL,2011-01-03,329.57001,16438280,329.70999,329.72000,325.64001,0.011315,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177.707767,0.595386
1,AAPL,2011-01-04,331.29001,11444072,331.29001,331.29999,332.44000,-0.001313,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178.826154,0.746331
2,AAPL,2011-01-05,334.00000,9486655,333.97000,334.00000,329.54999,0.005007,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329.709990,180.856263,0.566319
3,AAPL,2011-01-06,333.73001,11138250,333.64999,333.79001,334.71899,-0.002123,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330.027990,164.545590,0.394729
4,AAPL,2011-01-07,336.12000,11645048,336.12000,336.26001,333.98999,-0.001845,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330.822392,166.496499,0.711551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8488,MSFT,2022-03-25,303.67999,22544459,303.67999,303.69000,305.23001,0.005066,4543.060059,0.497449,...,0.018585,0.092213,0.104605,298.119195,295.624095,299.455198,16.187812,294.705144,-36.362854,0.542836
8489,MSFT,2022-03-28,310.70001,29560701,310.75000,310.76001,304.32999,0.007145,4575.520020,0.491188,...,0.022660,0.091832,0.388912,300.406616,296.994633,299.896171,16.502752,296.598117,-36.362854,0.542836
8490,MSFT,2022-03-29,315.41000,30349457,315.41000,315.42001,313.91000,0.012257,4631.600098,0.473442,...,0.006963,0.112858,0.895229,303.134504,298.668757,300.504556,17.965364,298.112496,-36.362854,0.542836
8491,MSFT,2022-03-30,313.85999,28134409,313.73999,313.85999,313.76001,-0.006294,4602.450195,0.500846,...,0.002373,0.127970,1.473633,305.084592,300.049778,301.028299,19.947739,300.641998,-36.362854,0.542836


In [ ]:
df4.to_csv('feature_eng_df.csv') # save the resulting df to csv